In [10]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve


The dataset contains two columns:
- category: the category of the email
- text: the text of the email

The dataset is available at [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection).

We will read the dataset into a pandas dataframe then specify the category column as the label and the text column as the feature.



In [11]:
df = pd.read_csv('data/spam.csv', encoding='ISO 8859-15')[['category', 'text']]
X, y = df['text'].values, df['category']


In [12]:
df.shape


(5572, 2)

We can either count the number of word occurences per email to vectorize the emails, or we can use the TFIDF algorithm to vectorize the emails. The 3000 best features are selected using the chi-squared test.

In [13]:
K = 3000
count = CountVectorizer(analyzer='word')
X = count.fit_transform(X).toarray()
X = SelectKBest(chi2, k=K).fit_transform(X, y)
X = pd.DataFrame(X, columns=count.get_feature_names_out()[0:K])


Now, we have a dataset with 3000 features, and we can use it to train a model.

In [14]:
X.shape


(5572, 3000)

In [15]:
pd.DataFrame(X).describe()


,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,ever,every,every1,everybody,everyboy,everyday,everyone,everyones,everyso,everythin
count,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,...,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000
mean,0.001795,0.005205,0.000359,0.000179,0.000179,0.000179,0.000359,0.001436,0.000538,0.000179,...,0.001256,0.024408,0.000179,0.000359,0.000179,0.001795,0.000179,0.004846,0.040201,0.000179
std,0.042330,0.076788,0.018944,0.013397,0.013397,0.013397,0.018944,0.037867,0.023199,0.013397,...,0.035425,0.188847,0.013397,0.018944,0.013397,0.046377,0.013397,0.079114,0.317331,0.013397
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,8.000000,1.000000


We will use the Naive Bayes classifier to train the model. We will use k-fold cross validation to test the model with k=10.

In [16]:
model = MultinomialNB(force_alpha=True, alpha=1)
kf = KFold(n_splits=10, random_state=1, shuffle=True)


In [31]:
def score_test(y_pred: np.ndarray, y_test):
    matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = matrix.ravel()
    return {
        'confusion_matrix': {'tn': int(tn), 'fp': int(fp), 'fn': int(fn), 'tp': int(tp)},
        'accuracy': round(accuracy_score(y_test, y_pred), 3),
        'f1': round(f1_score(y_test, y_pred, pos_label='ham'), 3),
        'recall': round(recall_score(y_test, y_pred, average="binary", pos_label='ham'), 3),
        'precision': round(precision_score(y_test, y_pred, pos_label='ham'), 3),
    }

For each fold of the cross validation, we will train the model on the training set and test the model on the test set. We will calculate the accuracy and the root mean squared of the results of each fold. Finally, we will calculate the average of the root mean squared of the results of each fold.

In [32]:
scores = {'accuracy': [], 'f1': [], 'recall': [], 'precision': []}
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[
        test_index], y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results = score_test(y_pred, y_test)
    scores['accuracy'] += [results['accuracy']]
    scores['f1'] += [results['f1']]
    scores['recall'] += [results['recall']]
    scores['precision'] += [results['precision']]
    print(results['confusion_matrix'])
print('Accuracy:', '%.3f +/- %.3f' %
      (np.mean(scores['accuracy']), np.std(scores['accuracy'])))
print('F1 Score:', '%.3f +/- %.3f' %
      (np.mean(scores['f1']), np.std(scores['f1'])))
print('Recall:', '%.3f +/- %.3f' %
      (np.mean(scores['recall']), np.std(scores['recall'])))
print('Precision:', '%.3f +/- %.3f' %
      (np.mean(scores['precision']), np.std(scores['precision'])))


{'tn': 484, 'fp': 6, 'fn': 4, 'tp': 64}
{'tn': 485, 'fp': 2, 'fn': 0, 'tp': 71}
{'tn': 474, 'fp': 4, 'fn': 5, 'tp': 74}
{'tn': 476, 'fp': 3, 'fn': 3, 'tp': 75}
{'tn': 477, 'fp': 2, 'fn': 2, 'tp': 76}
{'tn': 468, 'fp': 2, 'fn': 5, 'tp': 82}
{'tn': 478, 'fp': 1, 'fn': 1, 'tp': 77}
{'tn': 486, 'fp': 4, 'fn': 3, 'tp': 64}
{'tn': 485, 'fp': 2, 'fn': 1, 'tp': 69}
{'tn': 484, 'fp': 2, 'fn': 1, 'tp': 70}
Accuracy: 0.990 +/- 0.005
F1 Score: 0.995 +/- 0.003
Recall: 0.994 +/- 0.003
Precision: 0.995 +/- 0.004
